In [1]:
!/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip
!pip install tensorflow
!pip install keras

  Using cached pip-20.1.1-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
     |████████████████████████████████| 516.2 MB 9.4 kB/s  eta 0:00:01
     |████████████████████████████████| 104 kB 73.3 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 53.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 90 kB/s s eta 0:00:01
     |████████████████████████████████| 454 kB 43.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 102 kB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 155 k

In [2]:
!saved_model_cli show --dir "/inpbaseline-landmark-retrieval-model/baseline_landmark_retrieval_model" --all

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 1153, in main
    args.func(args)
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 714, in show
    _show_all(args.dir)
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 295, in _show_all
    tag_sets = saved_model_utils.get_saved_model_tag_sets(saved_model_dir)
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_utils.py", line 89, in get_saved_model_tag_sets
    saved_model = read_saved_model(saved_model_dir)
  File "/home/ec2-user/anaconda3/envs/amazonei_mxnet_p3

In [108]:
import numpy as np

import os
import cv2
import glob

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model, save_model
from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D
import keras.backend as K
from keras.models import Model, load_model
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input


In [109]:
train_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'train')
index_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'index')
test_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'test')

In [110]:
print(train_dir)

/GoogleLandmarkRetrieval2020/train


In [111]:
batch_size = 20
epochs = 1 
IMG_HEIGHT = 100
IMG_WIDTH = 100

In [112]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [122]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='train/0',
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')
print(train_data_gen)

Found 98637 images belonging to 16 classes.


In [123]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory='train/0/0',
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')


Found 6120 images belonging to 16 classes.


In [124]:
sample_training_images, _ = next(train_data_gen)

In [125]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [126]:
# plotImages(sample_training_images[:5])

In [127]:
model = Sequential([
#     Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
#     MaxPooling2D(),
#     Conv2D(32, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
#     Conv2D(64, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dense(1)
    VGG16(include_top = False, pooling = 'avg', weights = 'imagenet'),
    Dense(1, activation='softmax')
])
model.layers[0].trainable = False

In [128]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [129]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=98637 // batch_size,
    epochs=1,
    validation_data=val_data_gen,
    validation_steps=6120 // batch_size
)

 837/4931 [====>.........................] - ETA: 4:25:13 - loss: 0.0000e+00 - accuracy: 0.0705